# **Ingestion del archivo category.csv**

##### **1.- Leer archivo csv**

In [1]:
#p_file_date = "2025-07-07"
#p_environment = "Production"

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 3, Finished, Available, Finished)

In [2]:
%run configuration

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 4, Finished, Available, Finished)

In [3]:
%run common_functions

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 5, Finished, Available, Finished)

In [4]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 6, Finished, Available, Finished)

In [5]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
model_schema = StructType ( fields = [
    StructField("CategoryID", IntegerType(), False),
    StructField("DeviceCategory", StringType(), True)
])

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 7, Finished, Available, Finished)

In [6]:
# Ruta del archivo CSV en Azure Blob Storage o ADLS
abfs_path = f"{bronze_folder_path}/{p_file_date}/category.csv"

# Leer el archivo CSV desde ABFS
# El delimitador en este caso son de ; hay que agregar la option delimiter
df = spark.read.option("header", True) \
                .schema(model_schema) \
                .option("delimiter", ";") \
                .csv(abfs_path)

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 8, Finished, Available, Finished)

##### **2.- Cambiar nombres de columnas a lo requerido**

In [7]:
new_cols = ["category_id", "device_category"]

df_renamed = df.toDF(*new_cols)

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 9, Finished, Available, Finished)

##### **3.- Agregar columnas ingestion date y environment**

In [8]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df_renamed) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))


StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 10, Finished, Available, Finished)

In [9]:
%%sql
DROP TABLE if EXISTS category

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **4.- Pasar los archivos a delta a lh_silver**

In [10]:
df4.write.format("delta").mode("overwrite").saveAsTable("category")

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 12, Finished, Available, Finished)

In [11]:
%%sql
SELECT * from category
limit(10)

StatementMeta(, 6a24a66b-58e2-46cc-a73b-5d2b91a59373, 13, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 5 fields>